## Dataset Analysis

In [17]:
import sys
import os
cwd = os.getcwd()
parent_dir = os.path.abspath(os.path.join(os.path.dirname(cwd), '.')) # Get the absolute path to the parent directory
sys.path.insert(0, parent_dir) # Add the parent directory to sys.path

In [18]:
dataset_dir = parent_dir + "/data" # root directory where to download the datasets
batch_size = 16
dataset_dir

'/home/joshua/Documents/phd/CompressionVAEHSI/data'

In [19]:
# loading datamodule
from src.data.bigearth_datamodule import BigEarthDataModule
dm = BigEarthDataModule(dataset_dir=dataset_dir, batch_size=batch_size, bands='all')
dm.setup()

In [20]:
dm.setup_folds(5)
dm.setup_fold_index(0)

batch = next(iter(dm.train_dataloader()))

In [5]:
x, y = batch.values()

In [6]:
dm.maxs

tensor([6.0000e+00, 1.6000e+01, 9.8590e+03, 1.2872e+04, 1.3163e+04, 1.4445e+04,
        1.2477e+04, 1.2563e+04, 1.2289e+04, 1.5596e+04, 1.2183e+04, 9.4580e+03,
        5.8970e+03, 5.5440e+03])

In [14]:
# loading dataset
from src.data.bigearth_dataset import BigEarthDataset
data = BigEarthDataset(                                                                                                                                                                                                                         
    root=dataset_dir,
    bands='all',
    download=False,
)

In [21]:
print("Number of train samples: ", len(dm.train_dataloader().dataset))
print("Number of validation samples: ", len(dm.val_dataloader().dataset))
print("Number of test samples: ", len(dm.test_dataloader().dataset))

Number of train samples:  332342
Number of validation samples:  83086
Number of test samples:  103856


In [6]:
batch = next(iter(dm.train_dataloader()))
x, y = batch.values()

In [12]:
import numpy as np
np.random.seed(0)

In [15]:
num = np.random.random()

In [16]:
num

0.7151893663724195